# Earth System Tipping Cascade Exploration

**Author:** Jason Holt  
**Date:** 2025-12-06  
**Notebook:** 01_earth_system_tipping_exploration  

## Research Question

How do noise characteristics (Lévy vs Gaussian) and coupling strength affect cascade dynamics in a 4-element climate tipping system?

---

## Background

This notebook documents exploratory experiments with the PyCascades earth system model featuring four climate tipping elements:

| Element | Abbreviation | Timescale (years) | Tipping Threshold (°C) |
|---------|--------------|-------------------|------------------------|
| Greenland Ice Sheet | GIS | 98 | 2.24 |
| Thermohaline Circulation | THC | 6 | 4.69 |
| West Antarctic Ice Sheet | WAIS | 48 | 1.42 |
| Amazon Rainforest | AMAZ | 1 | 4.10 |

**Key Observation:** Timescales span two orders of magnitude - Amazon responds in ~1 year while Greenland takes ~98 years. This mismatch has implications for cascade propagation.

## Methods

### Base Configuration
- Global Mean Temperature (GMT): 2.5°C above pre-industrial
- Simulation duration: 100,000 years
- Network: Cusp-Cusp coupled system

### Experiments Conducted

| Exp | Noise Type | Alpha | Coupling | Noise Elements |
|-----|------------|-------|----------|----------------|
| 1 | Lévy | 1.5 | Default | GIS only |
| 2 | Lévy | 1.5 | Default | All elements |
| 3 | Gaussian | 2.0 | 0.50 | Realistic |
| 4 | Lévy | 1.5 | 0.25 | Realistic |
| 5 | Lévy | 1.5 | 0.50 | Realistic |

## Results

### Experiment 1: Original Settings (GIS noise only)

**Parameters:**
- `alphas = [1.5, 1.5, 1.5, 1.5]`
- `sigmas = [0.1, 0.0, 0.0, 0.0]` (only GIS receives noise)
- Default coupling strength

**Observations:**
- All four elements remain tightly coupled from t=0 to ~42,000 years
- At approximately t=42,000 years, GIS decouples and transitions to a lower state
- Other elements (THC, WAIS, AMAZ) remain in their original coupled state
- No cascade occurs despite GIS tipping

**Interpretation:**
The slow timescale of GIS (98 years) means its state change doesn't propagate quickly enough to trigger cascades in faster-responding elements.

---

### Experiment 2: All Elements with Lévy Noise

**Parameters:**
- `alphas = [1.5, 1.5, 1.5, 1.5]`
- `sigmas = [0.1, 0.1, 0.1, 0.1]` (all elements receive noise)

**Observations:**
- Highly chaotic dynamics
- AMAZ exhibits extreme spikes (up to -42 in some runs)
- Lévy noise heavy tails cause rare but extreme perturbations
- System shows no stable equilibrium

**Interpretation:**
Lévy noise with α=1.5 produces infinite variance - extreme events are expected. The fast Amazon timescale amplifies these perturbations.

---

### Experiment 3: Gaussian Noise with Stronger Coupling

**Parameters:**
- `alphas = [2.0, 2.0, 2.0, 2.0]` (Gaussian)
- Coupling strength: 0.50
- Realistic noise distribution

**Observations:**
- AMAZ tips independently to a higher state (~1.5)
- GIS, THC, WAIS remain coupled at lower state (~-1.0)
- No cascade propagation despite increased coupling
- Smooth, bounded fluctuations (no extreme spikes)

**Interpretation:**
Gaussian noise (α=2.0) has finite variance, producing bounded fluctuations. The timescale mismatch prevents cascade propagation even with 50% coupling strength. Amazon's 1-year response time is too fast to influence 48-98 year ice sheet dynamics.

---

### Experiment 4: Lévy Noise with Coupling 0.25

**Parameters:**
- `alphas = [1.5, 1.5, 1.5, 1.5]` (Lévy)
- Coupling strength: 0.25

**Observations:**
- GIS transitions to lower state around t=40,000 years (similar to Exp 1)
- AMAZ shows characteristic Lévy spikes (up to +8, down to -8)
- THC shows brief spikes but returns to baseline
- WAIS remains relatively stable
- Evidence of "back-tipping" - elements temporarily return to previous states

**Interpretation:**
Lévy noise can push elements across thresholds in both directions. The weak coupling (25%) is insufficient to lock in state changes.

---

### Experiment 5: Lévy Noise with Coupling 0.50

**Parameters:**
- `alphas = [1.5, 1.5, 1.5, 1.5]` (Lévy)
- Coupling strength: 0.50

**Observations:**
- Cleaner separation of states than Exp 4
- AMAZ stabilizes oscillating around ~1.5
- GIS, THC, WAIS stabilize around ~-1.0
- Less back-tipping than with 0.25 coupling

**Interpretation:**
Stronger coupling helps maintain state separation once elements have tipped. The system finds a quasi-stable configuration despite ongoing Lévy perturbations.

---

## Key Findings

### 1. Timescale Mismatch Prevents Cascades
The two-order-of-magnitude difference between Amazon (1 yr) and Greenland (98 yr) timescales creates a fundamental barrier to cascade propagation. Fast oscillations in Amazon don't persist long enough to influence slow ice sheet dynamics.

### 2. Lévy vs Gaussian Noise Produces Qualitatively Different Dynamics
- **Lévy (α=1.5):** Heavy-tailed, extreme events, back-tipping possible
- **Gaussian (α=2.0):** Bounded fluctuations, smoother trajectories

### 3. Coupling Strength Affects Stability, Not Cascade Initiation
Increasing coupling from 0.25 to 0.50 didn't trigger cascades but did help maintain state separation once elements had tipped independently.

### 4. GIS Decoupling at ~42k Years is Robust
Across multiple experiments, GIS consistently transitioned around t=40-42k years, suggesting this is an intrinsic property of the model configuration rather than a noise artifact.

---

## Questions for Further Investigation

1. **What coupling strength is required to overcome timescale mismatch?**
   - Systematically vary coupling from 0.1 to 1.0
   
2. **Can energy constraints create cascade-enabling synchronization?**
   - Core research question for Phase 3
   
3. **What is the critical GMT threshold for each element?**
   - Vary GMT systematically to map tipping boundaries
   
4. **How do asymmetric couplings affect dynamics?**
   - Current model uses symmetric coupling; real systems may be asymmetric

---

## Appendix: Code Reference

Base notebook: `example_earth_sys_levy_stable_noise.ipynb`

### Key Parameter Definitions

```python
# Timescales (years)
gis_time, thc_time, wais_time, amaz_time = 98.0, 6.0, 48.0, 1.0

# Tipping thresholds (°C above pre-industrial)
limits_gis = 2.24
limits_thc = 4.69  
limits_wais = 1.42
limits_amaz = 4.10

# Interaction probabilities (from Wunderling et al.)
pf_wais_to_gis = 0.193  # WAIS tipping increases GIS risk by 19.3%
pf_thc_to_gis = 0.578   # THC tipping increases GIS risk by 57.8%
pf_gis_to_thc = 0.066   # GIS tipping increases THC risk by 6.6%
pf_wais_to_thc = 0.047  # WAIS tipping increases THC risk by 4.7%
pf_gis_to_wais = 0.230  # etc.

# Noise parameters
alphas = [1.5, 1.5, 1.5, 1.5]  # Lévy stability parameter
sigmas = [0.1, 0.0, 0.0, 0.0]  # Noise strength per element
```

---

*End of exploration notebook. Next: Phase 3 energy-constrained extensions.*